# Automated HEC-RAS Analysis for Multiple AEP Events

This notebook demonstrates an end-to-end workflow for performing flood analysis with different Annual Exceedance Probability (AEP) events. We'll automate the following steps:

1. Generate hyetographs from NOAA Atlas 14 data for different AEP events
2. Download the Davis HEC-RAS project
3. Clone and configure HEC-RAS plans and unsteady flow files for each AEP event
4. Execute all plans in parallel
5. Extract and visualize results

## Required Libraries

We'll use the following libraries:
- `ras-commander`: For HEC-RAS automation
- `pandas`, `numpy`: For data manipulation
- `matplotlib`: For visualization
- Standard libraries: `os`, `re`, `pathlib`, etc.

Let's start by installing ras-commander (if needed) and importing necessary libraries.

In [1]:
# Install ras-commander if needed (uncomment to run)
# !pip install ras-commander

# Import necessary libraries
from ras_commander import *  # Import all ras-commander modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import shutil
from math import log, exp
from pathlib import Path
import time
import psutil  # For getting system CPU info
from IPython import display

# Configure matplotlib for better visualization
plt.style.use('ggplot')

# Set up logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Part 1: Generate Hyetographs from NOAA Atlas 14 Data

First, let's define functions to generate hyetographs from NOAA Atlas 14 precipitation frequency data. These functions will:
1. Parse duration strings from the CSV file
2. Read and process precipitation frequency data
3. Interpolate depths for each ARI (Annual Recurrence Interval)
4. Compute incremental depths
5. Apply the Alternating Block Method to generate hyetographs
6. Save the hyetographs to CSV files

In [2]:
# Function to parse duration strings and convert them to hours
def parse_duration(duration_str):
    """
    Parses a duration string and converts it to hours.
    Examples:
        "5-min:" -> 0.0833 hours
        "2-hr:" -> 2 hours
        "2-day:" -> 48 hours
    """
    match = re.match(r'(\d+)-(\w+):', duration_str.strip())
    if not match:
        raise ValueError(f"Invalid duration format: {duration_str}")
    value, unit = match.groups()
    value = int(value)
    unit = unit.lower()
    if unit in ['min', 'minute', 'minutes']:
        hours = value / 60.0
    elif unit in ['hr', 'hour', 'hours']:
        hours = value
    elif unit in ['day', 'days']:
        hours = value * 24
    else:
        raise ValueError(f"Unknown time unit in duration: {unit}")
    return hours

# Function to read and process the precipitation frequency CSV
def read_precipitation_data(csv_file):
    """
    Reads the precipitation frequency CSV and returns a DataFrame
    with durations in hours as the index and ARIs as columns.
    This function dynamically locates the header line for the data table.
    """
    with open(csv_file, 'r') as f:
        lines = f.readlines()

    header_line_idx = None
    header_pattern = re.compile(r'^by duration for ari', re.IGNORECASE)

    # Locate the header line
    for idx, line in enumerate(lines):
        if header_pattern.match(line.strip().lower()):
            header_line_idx = idx
            break

    if header_line_idx is None:
        raise ValueError('Header line for precipitation frequency estimates not found in CSV file.')

    # Extract the ARI headers from the header line
    header_line = lines[header_line_idx].strip()
    headers = [item.strip() for item in header_line.split(',')]
    
    if len(headers) < 2:
        raise ValueError('Insufficient number of ARI columns found in the header line.')

    aris = headers[1:]  # Exclude the first column which is the duration

    # Define the pattern for data lines (e.g., "5-min:", "10-min:", etc.)
    duration_pattern = re.compile(r'^\d+-(min|hr|day):')

    # Initialize lists to store durations and corresponding depths
    durations = []
    depths = {ari: [] for ari in aris}

    # Iterate over the lines following the header to extract data
    for line in lines[header_line_idx + 1:]:
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        if not duration_pattern.match(line):
            break  # Stop if the line does not match the duration pattern
        parts = [part.strip() for part in line.split(',')]
        if len(parts) != len(headers):
            raise ValueError(f"Data row does not match header columns: {line}")
        duration_str = parts[0]
        try:
            duration_hours = parse_duration(duration_str)
        except ValueError as ve:
            print(f"Skipping line due to error: {ve}")
            continue  # Skip lines with invalid duration formats
        durations.append(duration_hours)
        for ari, depth_str in zip(aris, parts[1:]):
            try:
                depth = float(depth_str)
            except ValueError:
                depth = np.nan  # Assign NaN for invalid depth values
            depths[ari].append(depth)

    # Create the DataFrame
    df = pd.DataFrame(depths, index=durations)
    df.index.name = 'Duration_hours'

    # Drop any rows with NaN values (optional, based on data quality)
    df = df.dropna()

    return df

# Function to perform log-log linear interpolation for each ARI
def interpolate_depths(df, total_duration):
    """
    Interpolates precipitation depths for each ARI on a log-log scale
    for each hour up to the total storm duration.
    """
    T = total_duration
    t_hours = np.arange(1, T+1)
    D = {}
    for ari in df.columns:
        durations = df.index.values
        depths = df[ari].values
        # Ensure all depths are positive
        if np.any(depths <= 0):
            raise ValueError(f"Non-positive depth value in ARI {ari}")
        # Log-log interpolation
        log_durations = np.log(durations)
        log_depths = np.log(depths)
        log_t = np.log(t_hours)
        log_D_t = np.interp(log_t, log_durations, log_depths)
        D_t = np.exp(log_D_t)
        D[ari] = D_t
    return D

# Function to compute incremental precipitation depths
def compute_incremental_depths(D, total_duration):
    """
    Computes incremental precipitation depths for each hour.
    I(t) = D(t) - D(t-1), with D(0) = 0.
    """
    incremental_depths = {}
    for ari, D_t in D.items():
        I_t = np.empty(total_duration)
        I_t[0] = D_t[0]  # I(1) = D(1) - D(0) = D(1)
        I_t[1:] = D_t[1:] - D_t[:-1]
        incremental_depths[ari] = I_t
    return incremental_depths

# Function to assign incremental depths using the Alternating Block Method
def assign_alternating_block(sorted_depths, max_depth, central_index, T):
    """
    Assigns incremental depths to the hyetograph using the Alternating Block Method.
    """
    hyetograph = [0.0] * T
    hyetograph[central_index] = max_depth
    remaining_depths = sorted_depths.copy()
    remaining_depths.remove(max_depth)
    left = central_index - 1
    right = central_index + 1
    toggle = True  # Start assigning to the right
    for depth in remaining_depths:
        if toggle and right < T:
            hyetograph[right] = depth
            right += 1
        elif not toggle and left >= 0:
            hyetograph[left] = depth
            left -= 1
        elif right < T:
            hyetograph[right] = depth
            right += 1
        elif left >= 0:
            hyetograph[left] = depth
            left -= 1
        else:
            print("Warning: Not all incremental depths assigned.")
            break
        toggle = not toggle
    return hyetograph

# Function to generate the hyetograph for a given ARI
def generate_hyetograph(incremental_depths, position_percent, T):
    """
    Generates the hyetograph for a given ARI using the Alternating Block Method.
    """
    max_depth = np.max(incremental_depths)
    incremental_depths_list = incremental_depths.tolist()
    central_index = int(round(T * position_percent / 100)) - 1
    central_index = max(0, min(central_index, T - 1))
    sorted_depths = sorted(incremental_depths_list, reverse=True)
    hyetograph = assign_alternating_block(sorted_depths, max_depth, central_index, T)
    return hyetograph

# Function to save the hyetograph to a CSV file
def save_hyetograph(hyetograph, ari, output_dir, position_percent, total_duration):
    """
    Saves the hyetograph to a CSV file.
    """
    df = pd.DataFrame({
        'Time_hour': np.arange(1, total_duration + 1),
        'Precipitation_in': hyetograph
    })
    filename = f'hyetograph_ARI_{ari}_years_pos{position_percent}pct_{total_duration}hr.csv'
    output_file = os.path.join(output_dir, filename)
    df.to_csv(output_file, index=False)
    logging.info(f"Hyetograph for ARI {ari} years saved to {output_file}")
    return output_file

## Generate Hyetographs for Different AEP Events

Now, let's use the functions defined above to generate hyetographs for different AEP events.
We'll download the NOAA Atlas 14 data for Davis, CA and generate hyetographs for various ARI values.

In [ ]:
# Open NOAA Atlas 14 data from CSV file
def open_noaa_atlas14_csv():
    """
    Opens NOAA Atlas 14 data from CSV file in data directory.
    """
    # Create data directory if it doesn't exist
    data_dir = Path('data')
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # Check if the file exists
    input_file = data_dir / 'PF_Depth_English_PDS_DavisCA.csv'
    if input_file.exists():
        logging.info(f"NOAA Atlas 14 data file found: {input_file}")
        return str(input_file)
    else:
        raise FileNotFoundError(f"NOAA Atlas 14 data file not found at {input_file}")

# Generate hyetographs
def generate_all_hyetographs(input_csv, output_dir, ari_values, position_percent=50, total_duration=24):
    """
    Generates hyetographs for specified ARI values.
    
    Parameters:
    - input_csv: Path to NOAA Atlas 14 CSV file
    - output_dir: Directory to save hyetographs
    - ari_values: List of ARI values to generate hyetographs for
    - position_percent: Position percentage for peak intensity (default: 50%)
    - total_duration: Total storm duration in hours (default: 24 hours)
    
    Returns:
    - Dictionary mapping ARI values to hyetograph file paths
    """
    # Ensure the output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    logging.info(f"Output directory is set to: {output_dir}")
    
    # Read precipitation data
    try:
        df = read_precipitation_data(input_csv)
        logging.info("Successfully read the input CSV file.")
    except Exception as e:
        logging.error(f"Error reading input CSV: {e}")
        raise
    
    # Display the first few rows of the DataFrame to verify
    logging.info("\nPrecipitation Frequency Data (first few rows):")
    display.display(df.head())
    
    # Interpolate depths
    try:
        D = interpolate_depths(df, total_duration)
        logging.info("Successfully interpolated precipitation depths.")
    except Exception as e:
        logging.error(f"Error during interpolation: {e}")
        raise
    
    # Compute incremental depths
    I = compute_incremental_depths(D, total_duration)
    logging.info("Successfully computed incremental depths.")
    
    # Generate and save hyetographs for each ARI
    hyetograph_files = {}
    for ari in ari_values:
        ari_str = str(ari)
        if ari_str in I:
            incremental_depths = I[ari_str]
            hyetograph = generate_hyetograph(incremental_depths, position_percent, total_duration)
            hyetograph_file = save_hyetograph(hyetograph, ari_str, output_dir, position_percent, total_duration)
            hyetograph_files[ari_str] = hyetograph_file
        else:
            logging.warning(f"ARI {ari} not found in the input data. Skipping.")
    
    logging.info(f"\nGenerated {len(hyetograph_files)} hyetographs.")
    return hyetograph_files

# Plot multiple hyetographs
def plot_multiple_hyetographs(aris, position_percent, total_duration, output_dir='hyetographs'):
    """
    Plots multiple hyetographs for specified ARIs on the same figure for comparison.
    
    Parameters:
    - aris (list of str or int): List of Annual Recurrence Intervals to plot
    - position_percent (int): Position percentage for the maximum intensity
    - total_duration (int): Total storm duration in hours
    - output_dir (str): Directory where hyetograph CSV files are saved
    
    Returns:
    - Plot object
    """
    plt.figure(figsize=(14, 7))
    
    for ari in aris:
        # Ensure ARI is a string for consistent filename formatting
        ari_str = str(ari)
        
        # Construct the filename based on the naming convention
        filename = f'hyetograph_ARI_{ari_str}_years_pos{position_percent}pct_{total_duration}hr.csv'
        filepath = os.path.join(output_dir, filename)
        
        # Check if the file exists
        if not os.path.exists(filepath):
            logging.warning(f"File '{filename}' does not exist in '{output_dir}'. Skipping.")
            continue
        
        # Read the hyetograph data
        try:
            hyetograph_df = pd.read_csv(filepath)
            logging.info(f"Successfully read hyetograph data from '{filename}'.")
        except Exception as e:
            logging.error(f"Error reading CSV file '{filename}': {e}")
            continue
        
        # Plot the hyetograph
        plt.bar(hyetograph_df['Time_hour'], hyetograph_df['Precipitation_in'], 
                width=0.8, edgecolor='black', alpha=0.5, label=f'ARI {ari_str} years')
    
    # Customize the plot
    plt.xlabel('Time (Hour)', fontsize=14)
    plt.ylabel('Incremental Precipitation (inches)', fontsize=14)
    plt.title(f'Comparison of Hyetographs for Different ARIs\nPosition: {position_percent}% | Duration: {total_duration} Hours', fontsize=16)
    plt.legend()
    plt.xticks(range(1, total_duration + 1, max(1, total_duration // 12)))
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Save the plot
    plt_filename = f"hyetograph_comparison_pos{position_percent}pct_{total_duration}hr.png"
    plt_filepath = os.path.join(output_dir, plt_filename)
    plt.savefig(plt_filepath, dpi=300)
    logging.info(f"Saved comparison plot to {plt_filepath}")
    
    return plt

# Run the hyetograph generation process
ari_values = [1, 2, 5, 10, 25, 50, 100, 200, 500, 1000]
position_percent = 50  # Position of peak intensity (50% = center of storm)
total_duration = 24    # Total storm duration in hours

# Open NOAA Atlas 14 data
input_csv = open_noaa_atlas14_csv()

# Generate hyetographs
output_dir = 'hyetographs'
hyetograph_files = generate_all_hyetographs(
    input_csv=input_csv,
    output_dir=output_dir,
    ari_values=ari_values,
    position_percent=position_percent,
    total_duration=total_duration
)

# Plot the hyetographs
plot = plot_multiple_hyetographs(
    aris=ari_values,
    position_percent=position_percent,
    total_duration=total_duration,
    output_dir=output_dir
)
plt.show()

## Part 2: Download and Prepare the Davis HEC-RAS Project

Now, let's download the Davis project using RasExamples.extract_project() and prepare it for our analysis. 
We'll then create a new folder for our AEP analysis to keep the original project intact.

In [ ]:
davis_path = RasExamples.extract_project("Davis")

# Create a new project folder for our AEP analysis
def create_aep_project_folder(original_project_folder, new_project_name):
    """
    Creates a new project folder for our AEP analysis.
    
    Parameters:
    - original_project_folder: Path to the original project folder
    - new_project_name: Name for the new project folder
    
    Returns:
    - Path to the new project folder
    """
    # Create path for the new project folder
    new_project_folder = original_project_folder.parent / new_project_name
    
    # Remove the new project folder if it already exists
    if new_project_folder.exists():
        logging.info(f"Removing existing folder: {new_project_folder}")
        shutil.rmtree(new_project_folder)
    
    # Copy the original project to the new folder
    logging.info(f"Copying project from {original_project_folder} to {new_project_folder}")
    shutil.copytree(original_project_folder, new_project_folder)
    
    logging.info(f"Created new project folder: {new_project_folder}")
    return new_project_folder

# Initialize the RAS project
def initialize_ras_project(project_folder, ras_version="6.6"):
    """
    Initializes the RAS project and returns the RAS object.
    
    Parameters:
    - project_folder: Path to the project folder
    - ras_version: HEC-RAS version (default: "6.6")
    
    Returns:
    - Initialized RAS object
    """
    try:
        ras_object = init_ras_project(project_folder, ras_version)
        logging.info(f"Initialized RAS project: {ras_object.project_name}")
        return ras_object
    except Exception as e:
        logging.error(f"Error initializing RAS project: {e}")
        raise

# Download the Davis project
davis_path = download_davis_project()

# Create a new project folder for our AEP analysis
aep_project_name = "Davis_AEP_Analysis"
aep_project_folder = create_aep_project_folder(davis_path, aep_project_name)

# Initialize the RAS project
ras_object = initialize_ras_project(aep_project_folder)

# Display project information
print("\nHEC-RAS Project Information:")
print(f"Project Name: {ras_object.project_name}")
print(f"Project Folder: {ras_object.project_folder}")

# Display available plans
print("\nAvailable Plans:")
display.display(ras_object.plan_df)

# Display available unsteady flow files
print("\nAvailable Unsteady Flow Files:")
display.display(ras_object.unsteady_df)

## Part 3: Clone Plans and Unsteady Flow Files for Each AEP Event

Now, let's clone Plan 02 for each AEP event and update the plan and unsteady flow files with the appropriate hyetograph data. This will allow us to simulate different AEP events with HEC-RAS.

In [ ]:
# Define the template plan and unsteady flow file numbers
template_plan = "02"  # Use plan 02 as the template
template_unsteady = "01"  # Use unsteady file 01 as the template

# Function to create plans and unsteady flow files for each AEP event
def create_aep_plans_and_unsteady_files(ras_object, template_plan, template_unsteady, ari_values, hyetograph_files):
    """
    Creates plans and unsteady flow files for each AEP event.
    
    Parameters:
    - ras_object: Initialized RAS object
    - template_plan: Plan number to use as template
    - template_unsteady: Unsteady flow file number to use as template
    - ari_values: List of ARI values to create plans for
    - hyetograph_files: Dictionary mapping ARI values to hyetograph file paths
    
    Returns:
    - Dictionary mapping ARI values to plan numbers
    - Dictionary mapping ARI values to unsteady flow file numbers
    """
    project_name = ras_object.project_name
    project_folder = ras_object.project_folder
    prj_file = ras_object.prj_file
    
    new_plan_numbers = {}
    new_unsteady_numbers = {}
    
    for ari in ari_values:
        ari_str = str(ari)
        if ari_str not in hyetograph_files:
            logging.warning(f"ARI {ari} does not have a hyetograph file. Skipping.")
            continue
        
        # Create new plan file by cloning template
        new_plan_number = RasPlan.clone_plan(template_plan, f"ARI_{ari}_years", ras_object)
        logging.info(f"Created new plan: {new_plan_number} for ARI {ari} years")
        new_plan_numbers[ari_str] = new_plan_number
        
        # Update plan description
        plan_description = f"Annual Exceedance Probability (AEP) Event\nAnnual Recurrence Interval (ARI): {ari} years\nExceedance Probability: {100/float(ari):.2f}%"
        # Get plan file path
        plan_file_path = RasPlan.get_plan_path(new_plan_number, ras_object)
        
        # Update the plan file directly since update_plan_value doesn't exist
        def update_description(lines):
            updated_lines = []
            in_description = False
            description_updated = False
            
            for line in lines:
                if line.strip().startswith("Description="):
                    updated_lines.append(f"Description={plan_description}\n")
                    description_updated = True
                    in_description = True
                elif in_description and line.strip() and not line.strip().startswith("Description="):
                    in_description = False
                    if not description_updated:
                        updated_lines.append(line)
                else:
                    updated_lines.append(line)
                    
            return updated_lines
        
        # Use RasUtils to update the file
        RasUtils.update_file(plan_file_path, update_description)
        logging.info(f"Updated plan description for ARI {ari} years")
        
        # Create new unsteady flow file by cloning template
        new_unsteady_number = RasPlan.clone_unsteady(template_unsteady, ras_object)
        logging.info(f"Created new unsteady flow file: {new_unsteady_number} for ARI {ari} years")
        new_unsteady_numbers[ari_str] = new_unsteady_number
        
        # Update unsteady flow file with hyetograph data
        unsteady_file_path = RasPlan.get_unsteady_path(new_unsteady_number, ras_object)
        
        # Read the hyetograph data
        hyetograph_file = hyetograph_files[ari_str]
        hyetograph_data = pd.read_csv(hyetograph_file)
        
        # Update flow title in unsteady file
        flow_title = f"ARI_{ari}_years"
        RasUnsteady.update_flow_title(unsteady_file_path, flow_title, ras_object)
        logging.info(f"Updated flow title for ARI {ari} years")
        
        # Apply the new unsteady file to the new plan
        RasPlan.set_unsteady(new_plan_number, new_unsteady_number, ras_object)
        logging.info(f"Applied unsteady flow file {new_unsteady_number} to plan {new_plan_number}")
    
    logging.info(f"Created {len(new_plan_numbers)} plans and {len(new_unsteady_numbers)} unsteady flow files.")
    return new_plan_numbers, new_unsteady_numbers

# Create plans and unsteady flow files for each AEP event
new_plan_numbers, new_unsteady_numbers = create_aep_plans_and_unsteady_files(
    ras_object=ras_object,
    template_plan=template_plan,
    template_unsteady=template_unsteady,
    ari_values=ari_values,
    hyetograph_files=hyetograph_files
)

# Display the new plans and unsteady flow files
print("\nNew Plans for AEP Events:")
for ari, plan_number in new_plan_numbers.items():
    print(f"ARI {ari} years: Plan {plan_number}")

print("\nNew Unsteady Flow Files for AEP Events:")
for ari, unsteady_number in new_unsteady_numbers.items():
    print(f"ARI {ari} years: Unsteady Flow {unsteady_number}")

# Refresh the RAS object to see the new plans and unsteady flow files
ras_object = initialize_ras_project(aep_project_folder)

# Display all plans
print("\nAll Plans:")
display.display(ras_object.plan_df)

# Display all unsteady flow files
print("\nAll Unsteady Flow Files:")
display.display(ras_object.unsteady_df)

## Part 4: Execute All Plans in Parallel

Now, let's execute all the plans we created in parallel using the RasCmdr.compute_parallel() function.
This will allow us to efficiently run multiple HEC-RAS simulations simultaneously.

In [ ]:
# Define a function to get the optimal number of workers based on system resources
def get_optimal_worker_count(cores_per_worker=2):
    """
    Calculate the optimal number of workers based on available physical cores.
    
    Parameters:
    - cores_per_worker: Number of cores to allocate to each worker (default: 2)
    
    Returns:
    - Optimal number of workers
    """
    # Get physical CPU cores
    physical_cores = psutil.cpu_count(logical=False)
    if physical_cores is None:
        physical_cores = psutil.cpu_count(logical=True) // 2  # Fallback estimate
    
    # Calculate optimal workers based on physical cores
    optimal_workers = physical_cores // cores_per_worker
    
    # Ensure at least 1 worker
    return max(1, optimal_workers)

# Execute plans in parallel
def execute_plans_in_parallel(ras_object, plan_numbers, compute_folder, cores_per_worker=2):
    """
    Executes multiple HEC-RAS plans in parallel.
    
    Parameters:
    - ras_object: Initialized RAS object
    - plan_numbers: List of plan numbers to execute
    - compute_folder: Folder to store computation results
    - cores_per_worker: Number of cores to allocate to each worker (default: 2)
    
    Returns:
    - Dictionary of execution results
    """
    # Check system resources
    cpu_count = psutil.cpu_count(logical=True)
    physical_cores = psutil.cpu_count(logical=False)
    memory_gb = psutil.virtual_memory().total / (1024**3)
    available_memory_gb = psutil.virtual_memory().available / (1024**3)
    
    logging.info(f"System Resources:")
    logging.info(f"- {physical_cores} physical CPU cores ({cpu_count} logical cores)")
    logging.info(f"- {memory_gb:.1f} GB total memory ({available_memory_gb:.1f} GB available)")
    
    # Calculate optimal number of workers
    max_workers = get_optimal_worker_count(cores_per_worker)
    logging.info(f"Using {max_workers} workers with {cores_per_worker} cores per worker")
    
    # Create compute folder if it doesn't exist
    compute_folder = Path(compute_folder)
    compute_folder.mkdir(parents=True, exist_ok=True)
    logging.info(f"Compute folder: {compute_folder}")
    
    # Record start time
    start_time = time.time()
    
    # Execute plans in parallel
    logging.info(f"Executing {len(plan_numbers)} plans in parallel...")
    results = RasCmdr.compute_parallel(
        plan_number=plan_numbers,
        max_workers=max_workers,
        num_cores=cores_per_worker,
        dest_folder=compute_folder,
        overwrite_dest=True,
        ras_object=ras_object
    )
    
    # Record end time and calculate duration
    end_time = time.time()
    total_duration = end_time - start_time
    
    logging.info(f"Parallel execution completed in {total_duration:.2f} seconds")
    
    return results

# Create compute folder
compute_folder = aep_project_folder.parent / "Davis_AEP_Compute"

# Get plan numbers to execute
plan_numbers_to_execute = list(new_plan_numbers.values())
print(f"Executing {len(plan_numbers_to_execute)} plans: {plan_numbers_to_execute}")

# Execute plans in parallel
execution_results = execute_plans_in_parallel(
    ras_object=ras_object,
    plan_numbers=plan_numbers_to_execute,
    compute_folder=compute_folder,
    cores_per_worker=2
)

# Create a DataFrame from the execution results for better visualization
results_df = pd.DataFrame([
    {"Plan": plan, "Success": success, "ARI": next((ari for ari, p in new_plan_numbers.items() if p == plan), None)}
    for plan, success in execution_results.items()
])

# Sort by ARI
results_df = results_df.sort_values("ARI", key=lambda x: x.astype(float))

# Display the results
print("\nExecution Results:")
display.display(results_df)

# Initialize a RAS project in the compute folder
compute_ras_object = initialize_ras_project(compute_folder)



## Part 5: Extract and Visualize Results

Finally, let's extract and visualize the results from the HEC-RAS simulations. We'll use the ras-commander library to extract water surface elevation (WSEL) data for a specific cell in the 2D mesh and create comparison plots.

## Summary

In this notebook, we've demonstrated an end-to-end workflow for performing flood analysis with different Annual Exceedance Probability (AEP) events using HEC-RAS and the ras-commander library. The key steps were:

1. **Generate Hyetographs**: Created precipitation hyetographs for different ARI values using NOAA Atlas 14 data
2. **Download and Prepare the HEC-RAS Project**: Downloaded the Davis project and created a new project folder for our analysis
3. **Clone and Configure Plans**: Created new plans and unsteady flow files for each AEP event and updated them with the appropriate hyetographs
4. **Execute Plans in Parallel**: Used parallel processing to efficiently run multiple HEC-RAS simulations simultaneously
5. **Extract and Visualize Results**: Extracted water surface elevation data for a specific cell and created comparison plots

This workflow demonstrates how the ras-commander library can be used to automate HEC-RAS workflows, making it easier to perform complex analyses with multiple scenarios. By using parallel processing, we can significantly reduce the overall computation time required for multi-scenario analyses.

### Next Steps

To build on this analysis, you could:
- Extract results for multiple cells or cross-sections to analyze spatial patterns of flooding
- Create flood extent maps for different AEP events
- Perform sensitivity analysis by varying parameters such as roughness or infiltration
- Compare results with observed flood data for model calibration
- Create animated visualizations of the flooding process

These extensions would further enhance the value of the analysis and provide more comprehensive insights into flood behavior and risk.